In [1]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
import pandas as pd
from keras import models, optimizers, backend
from keras.layers import core, convolutional, pooling, InputLayer
from sklearn import model_selection
#from preprocess.imageSteeringDB import ImageSteeringDB
import cv2
import random

local_project_path = r'C:/Users/Ketan/Thesis/PilotNet/src/driving_dataset/'
local_data_path = os.path.join(local_project_path, 'C:/Users/Ketan/Thesis/PilotNet/src/driving_dataset')
from tensorflow.keras.utils import Sequence
import matplotlib.image as mpimg

import os, json
import matplotlib.pyplot as plt

from PIL import Image
import skimage 
from skimage.transform import resize

from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from sklearn.preprocessing import LabelEncoder


from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from skimage.io import imread
from lime import lime_image

from keras.applications.imagenet_utils import decode_predictions




In [3]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [5]:
physical_device = tf.config.experimental.list_physical_devices('GPU')
print(f'Device found : {physical_device}')

Device found : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
tf.config.experimental.get_memory_growth(physical_device[0])


In [7]:
# tf.config.experimental.set_memory_growth(physical_device[0],True)


In [8]:
# tf.debugging.set_log_device_placement(True)

# # Place tensors on the CPU
# with tf.device('/CPU:0'):
#     start_time = time.time()
#     a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
#     b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

# # Run on the GPU
# c = tf.matmul(a, b)
# timer = time.time()
# print(timer - start_time)
# print(c)

In [2]:
dataset = ('C:/Users/Ketan/Thesis/PilotNet/src/driving_dataset')

In [3]:
# Read the data
df = pd.io.parsers.read_csv(os.path.join(dataset, 'data_new.csv'))
# Split data into training and validation sets
df_train, df_valid = model_selection.train_test_split(df, test_size=.2)

In [4]:
x_train = df['image']
y_train = df['angle']

In [5]:
pd.options.mode.chained_assignment = None #default = warn

In [6]:
df_train['angle_range'] = pd.qcut(df_train['angle'], q=10)
df_valid['angle_range'] = pd.qcut(df_valid['angle'], q=10)

In [7]:
df_train['labels']= df_train['angle_range'].cat.codes
df_valid['labels']= df_valid['angle_range'].cat.codes

In [8]:
df_train['labels'] = df_train['labels'].apply(str)
df_valid['labels'] = df_valid['labels'].apply(str)


In [23]:
len(df_train)

36324

In [16]:
# df_train.angle_range
#Categories (10, interval[float64]): [(-159.901, -18.76] < (-18.76, -9.48] < (-9.48, -2.52] < (-2.52, 0.0] ... (2.42, 6.25] 
                                    #< (6.25, 11.5] < (11.5, 22.08] < (22.08, 501.78]]

In [27]:
datagen=ImageDataGenerator(rescale=1./255)
train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory=dataset, x_col="image", y_col="labels", 
                                            class_mode="categorical", target_size=(256,455), batch_size=32)

Found 36324 validated image filenames belonging to 10 classes.


In [29]:
len(valid_generator)

284

In [10]:
valid_generator=datagen.flow_from_dataframe(dataframe=df_valid, directory=dataset, 
                                            x_col="image", y_col="labels", class_mode="categorical", target_size=(256,455), 
                                            batch_size=32)

Found 9081 validated image filenames belonging to 10 classes.


In [11]:
from sklearn import decomposition 

In [12]:
pca = decomposition.PCA()

In [13]:
pca.n_components = 2

In [32]:
# pca_data = pca.fit_transform()

In [19]:
from tensorflow.keras.applications.resnet50 import ResNet50


In [19]:
from keras.applications.vgg16 import VGG16


In [58]:
base_model2 = tf.keras.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(256,455,3),
    pooling=None,
    classes=1000,
)

In [60]:
base_model2.trainable = False

In [61]:
base_model2.summary()

Model: "densenet201"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 256, 455, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 262, 461, 3)  0           input_10[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 228, 64) 9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 228, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

conv4_block27_1_relu (Activatio (None, 16, 28, 128)  0           conv4_block27_1_bn[0][0]         
__________________________________________________________________________________________________
conv4_block27_2_conv (Conv2D)   (None, 16, 28, 32)   36864       conv4_block27_1_relu[0][0]       
__________________________________________________________________________________________________
conv4_block27_concat (Concatena (None, 16, 28, 1120) 0           conv4_block26_concat[0][0]       
                                                                 conv4_block27_2_conv[0][0]       
__________________________________________________________________________________________________
conv4_block28_0_bn (BatchNormal (None, 16, 28, 1120) 4480        conv4_block27_concat[0][0]       
__________________________________________________________________________________________________
conv4_block28_0_relu (Activatio (None, 16, 28, 1120) 0           conv4_block28_0_bn[0][0]         
__________

conv5_block21_1_bn (BatchNormal (None, 8, 14, 128)   512         conv5_block21_1_conv[0][0]       
__________________________________________________________________________________________________
conv5_block21_1_relu (Activatio (None, 8, 14, 128)   0           conv5_block21_1_bn[0][0]         
__________________________________________________________________________________________________
conv5_block21_2_conv (Conv2D)   (None, 8, 14, 32)    36864       conv5_block21_1_relu[0][0]       
__________________________________________________________________________________________________
conv5_block21_concat (Concatena (None, 8, 14, 1568)  0           conv5_block20_concat[0][0]       
                                                                 conv5_block21_2_conv[0][0]       
__________________________________________________________________________________________________
conv5_block22_0_bn (BatchNormal (None, 8, 14, 1568)  6272        conv5_block21_concat[0][0]       
__________

Total params: 18,321,984
Trainable params: 0
Non-trainable params: 18,321,984
__________________________________________________________________________________________________


In [20]:
base_model = ResNet50(weights='imagenet',include_top=False)
        

In [21]:
base_model.trainable = False

In [ ]:
base_model.summary()

In [22]:
base_model1 = VGG16(weights='imagenet',input_shape=(256,455,3),include_top=False)


In [23]:
base_model1.trainable = False

In [ ]:
base_model1.summary()

In [ ]:
inp = Input(shape=(256,455, 3), name='image_input')

vgg_model = base_model1
vgg_model.trainable = False

x = core.Flatten()(vgg_model)
x = core.Dense(512, activation='relu', name='fc1')(x)
x = core.Dense(512, activation='relu', name='fc2')(x)
x = core.Dense(10, activation='softmax', name='predictions')(x)
new_model = models.Model(inputs=inp, outputs=x)
new_model.compile(optimizer='adam', loss='categorical_crossentropy', 
               metrics=['accuracy'])

In [29]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

Total params: 23,587,712
Trainable params: 0
Non-trainable params: 23,587,712
__________________________________________________________________________________________________


In [79]:
model = models.Sequential()
# model.add(InputLayer(input_shape=(128,380, 3)))
model.add(base_model2)
model.add(pooling.MaxPooling2D())
model.add(core.Flatten())
model.add(core.Dense(256, activation='relu'))
model.add(core.Dropout(.5))
model.add(core.Dense(128, activation='relu'))
# model.add(core.Dropout(.5))
# model.add(core.Dense(10, activation='relu'))
model.add(core.Dense(10, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-04), loss='categorical_crossentropy',metrics=['AUC', 'categorical_accuracy'],)

In [80]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, 8, 14, 1920)       18321984  
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 7, 1920)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 53760)             0         
_________________________________________________________________
dense_28 (Dense)             (None, 256)               13762816  
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_30 (Dense)             (None, 10)              

In [81]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

In [82]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [83]:
checkpoint_filepath = './logs'


In [84]:
callback1 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', verbose=0, 
                                               save_best_only=False, save_weights_only=False, mode='auto', 
                                               save_freq='epoch',options=None)

In [85]:
with tf.device('/GPU:0'):

    model.fit_generator(generator=train_generator,
                        steps_per_epoch=STEP_SIZE_TRAIN,
                        validation_data=valid_generator,
                        validation_steps=STEP_SIZE_VALID,
                        epochs=10, callbacks=[callback, callback1])

Epoch 1/10


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  Input to reshape is a tensor with 614400 values, but the requested shape requires a multiple of 53760
	 [[node sequential_10/flatten_8/Reshape (defined at \AppData\Local\Temp/ipykernel_2516/1795674310.py:3) ]]
	 [[assert_less_equal/Assert/AssertGuard/pivot_f/_13/_41]]
  (1) Invalid argument:  Input to reshape is a tensor with 614400 values, but the requested shape requires a multiple of 53760
	 [[node sequential_10/flatten_8/Reshape (defined at \AppData\Local\Temp/ipykernel_2516/1795674310.py:3) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_125439]

Function call stack:
train_function -> train_function
